In [1]:
#Predict reactivity of a protein-ligand complex
#Before running, download results directory into pred/script/pafnucy/
    #https://gitlab.com/cheminfIBB/pafnucy/-/tree/master/results 
import os
import sys
import pandas as pd
import numpy as np
import pickle
from openbabel import pybel

In [2]:
#Arguments
protein = 'tropb'
ligand = '2'
pdb_with_fad_path = f'../../cofactor/pdb_with_fad/{protein}_fad.pdb'
dockdir = f'../../dock/poses/{protein}_{ligand}_prot'
stereo_df = pd.read_excel(f'../stereo/{protein}_{ligand}_stereo.xlsx', header = 0, index_col = 0)

In [3]:
#Read trained logistic regressor model for consensus reactivity
log_reg = pickle.load(open(f'../../si_data/log_reg_models/prot_0.75_fftdock_3_logreg.pkl', 'rb'))
log_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             Conversion   No. Observations:                   67
Model:                          Logit   Df Residuals:                       63
Method:                           MLE   Df Model:                            3
Date:                Fri, 07 Apr 2023   Pseudo R-squ.:                  0.2368
Time:                        13:40:15   Log-Likelihood:                -34.980
converged:                       True   LL-Null:                       -45.835
Covariance Type:            nonrobust   LLR p-value:                 7.501e-05
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Avg Fad Angle                     0.1035      0.043      2.393      0.017       0.019       0.188
Avg Anion Dist                   -0.4565      0.199     -2.293      0.022      -0.847      -0.066
Avg Docking Energy Efficiency    -1.3699      0.389     -3.519      0.000      -2.133      -0.607
Avg Pafnucy pKa Efficiency      -16.6168      5.488     -3.028      0.002     -27.373      -5.860
=================================================================================================
"""

In [4]:
#Angle, anion dist and docking energy from stereochemistry analysis of cluster df
stereo_df

,cluster,size,min_ener,min_index,average energy,std energy,stereo,angle,dist_to_fad,anion_dist
0,"['116', '121', '160', '162', '166', '185', '19...",31,-52.23,340,-26.32,15.43,R,110.67,5.781542,4.587086
1,"['105', '106', '132', '146', '153', '176', '19...",18,-50.55,132,-22.22,24.09,R,108.20,5.683485,4.820822
2,"['123', '228', '241', '246', '284', '297', '30...",16,-42.00,420,-29.00,10.68,R,116.33,5.553344,4.595473
3,"['131', '140', '147', '164', '173', '188', '28...",23,-40.47,344,-13.77,21.18,R,132.26,4.742048,3.128783
4,"['315', '376']",2,-35.26,376,-25.86,9.39,S,48.58,6.074435,11.829259
...,...,...,...,...,...,...,...,...,...,...
94,"['145', '223', '27', '28', '47']",5,409.14,47,431.31,11.99,S,77.36,6.697443,3.234980
95,['45'],1,445.52,45,445.52,0.00,S,59.22,6.822106,3.230141
96,"['363', '497']",2,1978.06,497,3924.27,1946.20,S,33.15,2.756856,8.899055
97,['299'],1,5066.50,299,5066.50,0.00,S,52.86,2.489911,8.118101


In [5]:
#Get predicted pKd from Pafnucy
#Use top docked pose for protein-ligand complex
top_index = stereo_df['min_index'].iloc[0]
top_pose_path = os.path.join(dockdir, f'{protein}_{ligand}_{top_index}.pdb')

In [6]:
#Prepare protein ligand complex
complex_file = f'../pafnucy/{protein}_{ligand}.hdf'
prepare_cmd = f'{sys.executable} ./pafnucy/prepare.py -l {top_pose_path} --ligand_format pdb ' + \
f'-p {pdb_with_fad_path} --pocket_format pdb -o {complex_file}'
os.system(prepare_cmd)

1 ligands and 1 pockets to prepare:
 pocket: /home/azamh/demo/seq_struct_func/cofactor/pdb_with_fad/tropb_fad.pdb
 ligand: /home/azamh/demo/seq_struct_func/dock/poses/tropb_2_prot/tropb_2_340.pdb



reading /home/azamh/demo/seq_struct_func/dock/poses/tropb_2_prot/tropb_2_340.pdb


created /home/azamh/demo/seq_struct_func/pred/pafnucy/tropb_2.hdf with 1 structures


0

In [7]:
#Predict with trained CNN
prediction_file = f'../pafnucy/{protein}_{ligand}.csv'
network = './pafnucy/results/batch5-2017-06-05T07:58:47-best'
predict_cmd = f'{sys.executable} ./pafnucy/predict.py -i {complex_file} -n {network} -o {prediction_file}'
os.system(predict_cmd)

2023-04-07 13:40:17.030735: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-07 13:40:17.112917: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Instructions for updating:
non-resource variables are not supported in the long term
2023-04-07 13:40:19.935029: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  

loaded 1 complexes

restored network from /home/azamh/demo/seq_struct_func/pred/script/pafnucy/results/batch5-2017-06-05T07:58:47-best

20 samples per batch

results saved to /home/azamh/demo/seq_struct_func/pred/pafnucy/tropb_2.csv


0

In [8]:
pred_pkd = pd.read_csv(prediction_file, header = 0, index_col = 0)['prediction'].iloc[0]
pred_pkd

6.3208246

In [9]:
#Process values for logistic regression model

#Dictionary for ligand heavy atom counts:
ha_dict = {'2':17, '3':18, '4':17, '5':14}

#Read in pose geometric information from stereo_df
top_cluster = stereo_df.iloc[0]
dock_energy = top_cluster['min_ener']
angle = top_cluster['angle']
if angle > 90:
    angle = 180 - angle
anion_dist = top_cluster['anion_dist']

#Rescale by heavy atom counts for pkd and dock energy
dock_energy_efficiency = dock_energy / ha_dict[ligand]
pred_pkd_efficiency = pred_pkd / ha_dict[ligand]

print('dock_energy_efficiency', dock_energy_efficiency, 'kcal/mol/HA')
print('angle', angle, 'degrees')
print('anion_dist', anion_dist, 'angstrom')
print('pred_pkd_efficiency', pred_pkd_efficiency, '1/HA')

dock_energy_efficiency -3.0723529411764705 kcal/mol/HA
angle 69.33 degrees
anion_dist 4.58708589411622 angstrom
pred_pkd_efficiency 0.3718132117647059 1/HA


In [10]:
#Predict with logistic regression model: 0 -> unreactive, 1 -> reactive
log_reg.predict([angle, anion_dist, dock_energy_efficiency, pred_pkd_efficiency])

array([0.95748382])